## Imports

In [1]:
# Connecting to the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
## Importing all dependancies
import pandas as pd
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, Dataset
import tqdm
import torch.optim as optim

In [3]:
## Accessing KG to secondary model map of users
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/user_kg_index_secondary_id_map_tucson','rb') as f:
  user_kg_index_secondary_id_map = pickle.load(f)

In [4]:
## Accessing KG to secondary model map of items
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/item_kg_index_secondary_id_map_tucson','rb') as f:
  item_kg_index_secondary_id_map = pickle.load(f)

In [5]:
## Accessing KG Node map dict
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_node_map_dict','rb')as f:
  KG_node_map_dict = pickle.load(f)

In [6]:
## Accessing trained secondary model
## Accessing Trained Secondary Model

# Define the model architecture
class CFNet(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CFNet, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim, _weight=user_embeddings)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim, _weight=item_embeddings)
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embeddings(user_ids)
        item_embeds = self.item_embeddings(item_ids)
        x = torch.cat([user_embeds, item_embeds], dim=1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/secondary_model_tucson', 'rb') as handle:
    secondary_model = pickle.load(handle)

## Definations

In [7]:
# Getting predicted rating of POI for user
def get_predicted_rating (user,poi):

  ## Script for poi rating prediction for a user from secondary model

  user_id = user_kg_index_secondary_id_map[KG_node_map_dict[user]]
  item_id = item_kg_index_secondary_id_map[KG_node_map_dict[poi]]

  user_tensor = torch.LongTensor([user_id])
  item_tensor = torch.LongTensor([item_id])

  user_embed = secondary_model.user_embeddings(user_tensor)
  item_embed = secondary_model.item_embeddings(item_tensor)

  x = torch.cat([user_embed, item_embed], dim=1)

  x = nn.functional.relu(secondary_model.fc1(x))
  x = secondary_model.fc2(x)

  #rating_pred = float(x)
  user_tensor.detach()
  item_tensor.detach()
  user_embed.detach()
  item_embed.detach()
  x.detach()
  x.detach()
  return x

In [8]:
# Group embedding learning model
class RatingPredictionModel(nn.Module):
    def __init__(self, users_embed_dim, pois_embed_dim,num_users):
        super(RatingPredictionModel, self).__init__()
        self.weights = nn.Parameter(torch.randn(num_users))
        self.fc1 = nn.Linear(users_embed_dim + pois_embed_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, group_embedding, poi_embeddings):
        user_weights = torch.softmax(self.weights, dim=0)
        weighted_user_embeddings = [weight * embedding for weight, embedding in zip(user_weights, group_embedding)]
        group_embedding = torch.stack(weighted_user_embeddings, dim=0).sum(dim=0)
        temp_grp_embed = group_embedding.repeat(len(grp_previsited_pois),1)
        x = torch.cat((temp_grp_embed, poi_embeddings), dim=1)
        x = nn.functional.relu(secondary_model.fc1(x))
        x = secondary_model.fc2(x)
        return x

## Pipeline

In [9]:
## Accessing User-POI visit train dictonary
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/UP_dict_train','rb')as f:
  UP_dict_train = pickle.load(f)

## Creating list of unique POIs in train dictonary
train_POI_list = []
for user in UP_dict_train:
  for record in UP_dict_train[user]:
    train_POI_list.append(record[0])
train_POI_list = list(set(train_POI_list))

In [19]:
# Group embedding and user weights learning loop
group_embeddings = []
group_user_weights = []

# Accessing groups
import pickle
file = open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_group_10', "rb")
user_group = pickle.load(file)

for group in tqdm.tqdm(user_group):

  #Getting List of POIs visited by group members
  #grp_previsited_pois = get_group_previsited_pois (group)
  grp_previsited_pois = train_POI_list
  #Get group member user embedding
  user_embeddings = []
  for user in group:
    user_id = user_kg_index_secondary_id_map[KG_node_map_dict[user]]
    user_tensor = torch.LongTensor([user_id])
    user_embed = secondary_model.user_embeddings(user_tensor)[0]
    res_embed = user_embed.clone()
    user_embeddings.append(res_embed.detach())


  #Get POI embedding of previsited POIs
  poi_embeddings = []
  for poi in grp_previsited_pois:
    item_ind = item_kg_index_secondary_id_map[KG_node_map_dict[poi]]
    item_ind_tensor = torch.LongTensor([item_ind])
    item_embed = secondary_model.item_embeddings(item_ind_tensor)[0]
    res_embed = item_embed.clone()
    poi_embeddings.append(res_embed.detach())
  poi_embeddings = torch.stack(poi_embeddings,dim=0)


  #Initializing group embedding and fetching ground truth for group
  group_embedding = torch.stack(user_embeddings,dim=0)
  group_ratings = []
  individual_poi_ratings = []
  for poi in grp_previsited_pois:
    for user in group:
      temp_rating = float(get_predicted_rating (user,poi))
      individual_poi_ratings.append(temp_rating)
    group_ratings.append(torch.tensor(sum(individual_poi_ratings)/len(individual_poi_ratings)))
  ground_truth_ratings = torch.stack(group_ratings)


  #Initialising model for group
  model = RatingPredictionModel(user_embeddings[0].shape[0], poi_embeddings.shape[1],len(user_embeddings))


  #Defining optimizer and loss function
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.05)


  #Training loop for group
  num_epochs = 100  # Adjust the number of epochs as needed
  for epoch in range(num_epochs):
      # Zero the gradients
      optimizer.zero_grad()
      # Forward pass
      output = model(group_embedding, poi_embeddings)
      output_reshape = output.view(len(grp_previsited_pois))
      # Calculate the loss
      loss = criterion(output_reshape, ground_truth_ratings)
      # Backward pass and optimization
      loss.backward()
      optimizer.step()


  #Getting final weights and generating final group embedding
  user_weights = torch.softmax(model.weights, dim=0)
  weighted_user_embeddings = [weight * embedding for weight, embedding in zip(user_weights, group_embedding)]
  group_embedding = torch.stack(weighted_user_embeddings, dim=0).sum(dim=0)


  #Saving final weights and embeddings
  group_embeddings.append(group_embedding)
  group_user_weights.append(model.weights)

100%|██████████| 127/127 [39:18<00:00, 18.57s/it]


In [20]:
# Saving the group embeddings and user weights
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/group10_embeddings_tucson','wb')as f:
  pickle.dump(group_embeddings,f)

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/group10_member_weights_tucson','wb')as f:
  pickle.dump(group_user_weights,f)

## Inferencing